# CNN MNIST

In [1]:
#importing functions from python3 to python2
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#importing numpy and tensorflow
import numpy as np
import tensorflow as tf

#ignore all the warnings and don't show them in the notebook
import warnings
warnings.filterwarnings('ignore')

#treshold on what messages are to be logged
tf.logging.set_verbosity(tf.logging.INFO)
#importing debug library
from tensorflow.python import debug as tf_debug

## Debugger

### Uncomment the below line and execute the code to run the debugger.

### Go to the link once you start execution    			http://localhost:6006/

In [2]:
#Uncomment the below line to run the debugger
#Add monitor=[hook] as a parameter to the estimators below
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
import wget

In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
            inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }
    #if predict mode, run the estimator and return the values
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
def main(unused_argv):
    # Load training data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images  # Returns np.array
    #load the the train data labels
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    #load the test data
    eval_data = mnist.test.images  # Returns np.array
    #load the test data labels
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)


In [5]:
#Run the model, steps set to 200 instead of 20000 as the execution time was large
#Changing steps back to 20000 in model training results in an accuracy of 97%
if __name__ == "__main__":
    tf.app.run()


Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_ch

INFO:tensorflow:loss = 2.2763422, step = 201
INFO:tensorflow:probabilities = [[0.10545531 0.10635927 0.08974896 0.09320334 0.09953374 0.097153
  0.09589889 0.10162999 0.10915956 0.10185806]
 [0.11278766 0.09426642 0.08174032 0.09262894 0.09326299 0.08982173
  0.10270584 0.10893599 0.11982785 0.10402226]
 [0.10936493 0.10658029 0.11176837 0.09389289 0.08601717 0.08918862
  0.10797025 0.0963737  0.1027183  0.09612542]
 [0.10888703 0.10075003 0.10114063 0.0958903  0.08485499 0.09934626
  0.10826171 0.085552   0.10900743 0.10630959]
 [0.10177691 0.10508612 0.09357838 0.10123185 0.08779202 0.10340195
  0.09398383 0.10894009 0.10391164 0.10029725]
 [0.10816914 0.11322699 0.10703857 0.09258401 0.08966339 0.0924964
  0.10121699 0.09741224 0.0973569  0.1008353 ]
 [0.1227513  0.10212713 0.10438173 0.09432421 0.09563052 0.09459161
  0.09668764 0.09752382 0.09950272 0.09247933]
 [0.10767598 0.10236245 0.10312651 0.10318874 0.09048291 0.0990045
  0.09313032 0.09713583 0.10383689 0.10005589]
 [0.100

INFO:tensorflow:global_step/sec: 0.504196
INFO:tensorflow:probabilities = [[0.10609921 0.10167987 0.09553295 0.10301723 0.11116837 0.09295989
  0.09588337 0.09783461 0.09655168 0.0992728 ]
 [0.10660495 0.1065634  0.09514769 0.10147663 0.100284   0.09970698
  0.09086689 0.09753104 0.10508293 0.0967355 ]
 [0.11160638 0.09967358 0.09737933 0.10739765 0.08935992 0.08753562
  0.09823395 0.10109878 0.10971569 0.09799904]
 [0.11102714 0.09104878 0.10521723 0.09972674 0.09033643 0.10078835
  0.10308491 0.09605205 0.10563922 0.09707918]
 [0.10630025 0.09534957 0.09551922 0.10356397 0.09385126 0.09387089
  0.10928044 0.09976465 0.10531443 0.09718534]
 [0.11871187 0.10509591 0.11029071 0.10791778 0.0950431  0.08332752
  0.09546831 0.09058501 0.0940613  0.09949842]
 [0.09756466 0.10326674 0.09717186 0.09813087 0.10941326 0.09593491
  0.09230756 0.10461731 0.09651837 0.10507445]
 [0.12584518 0.09925114 0.09918955 0.11054649 0.09276852 0.09123744
  0.09690267 0.08042741 0.1050038  0.09882773]
 [0.12

INFO:tensorflow:loss = 2.229753, step = 301 (198.352 sec)


KeyboardInterrupt: 